Final Project Submission <br>
Student name: Qilun Chen, Evan Serrano<br>
Student pace: full time<br>
Scheduled project review date/time: April/1/2022<br>
Instructor name: Praveen Gowtham, Joe Comeaux<br>
Blog post URL:https://github.com/nkbuddy/dsc-phase-3-project-NBA

### Table of Contents
* [STEP 1: Define the Problem](#Define-the-Problem)
* [Step 2: Gather the Data](#Gather-the-Data)
* [Step 3: Prepare Data for Consumption](#Prepare-Data-for-Consumption)
    * [3.1 Import Libraries](#Import-Libraries)
        * [3.11 Load Data Modelling Libraries](#Load-Data-Modelling-Libraries)
    * [3.2 Meet and Greet Data](#Meet-and-Greet-Data)
        * [3.21 The 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting](#4C)
        * [3.22 Clean Data](#Clean-Data)
        * [3.23 Convert Formats](#Convert-Formats)
        * [3.24 Da-Double Check Cleaned Data](#Da-Double-Check-Cleaned-Data)
        * [3.25 Split Training and Testing Data](#Split-Training-and-Testing-Data)
* [Step 4: Perform Exploratory Analysis with Statistics](#Perform-Exploratory-Analysis-with-Statistics)
* [Step 5: Model Data](#Model-Data)
    * [5.1 Evaluate Model Performance](#Evaluate-Model-Performance)
        * [5.11 Model Performance with Cross-Validation (CV)](#CV)
        * [5.12 Tune Model with Hyper-Parameters](#Tune-Model-with-Hyper-Parameters)
        * [5.13 Tune Model with Feature Selection](#Tune-Model-with-Feature-Selection)
* [Step 6: Validate and Implement](#Validate-and-Implement)
* [STEP 7: Optimize and Strategize](#Optimize-and-Strategize)

# STEP 1: Define the Problem <a class="anchor" id="Define-the-Problem"></a>

Giving a recommendation to people who has favortie coffee but wants to try sometings news.

# Step 2: Gather the Data <a class="anchor" id="Gather-the-Data"></a>

Web scraping from coffeereview.com

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import numpy as np
import PIL.ImageFile as ImageFile
import random

In [44]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
url = 'https://www.coffeereview.com/review/?locations=na'
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")
containers = page_soup.findAll("p","review-title")
for container in containers:
    print(container.text)

Costa Rica Naranjo Danilo Salazar Arias
Kenya Kahunyo AA
Honduras COMSA Oscar Omar Alonzo
Rwanda Gasharu Natural
Yemen Haraaz Red Mahal Aqeeq ul Station Natural
Sítio Pedra Menina
Brazil Sítio Ponte Fazenda Ponte
Brazil Legender Sitio Taquara Natural
Ethiopia Odola Washed
Minanga Village Sulawesi
Ka‘ū Tropical Punch Washed
Birambo Village DR Congo
Kenya Kamgogo Small Lot Reserve
Ethiopia Belayneh Bariso Natural
Kuta Kofi Papua New Guinea
Rwanda Gatare Natural
Rock the House Blend
Kona SL34 Champagne Natural Uluwehi Farm
Yemen Al Mashtal Al Burhani
Ka‘ū Lactic Natural


In [61]:
url_each_coffee = []
for i in range(1,289):
    if i == 1:
        url = "https://www.coffeereview.com/review/?locations=na"
    else:
        url="https://www.coffeereview.com/review/page/"+str(i)+"/?locations=na"
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    containers = page_soup.findAll("p","review-title")
    for container in containers:
        url_each_coffee.append(container.a['href'])

In [62]:
url_each_coffee[0]

'https://www.coffeereview.com/review/costa-rica-naranjo-danilo-salazar-arias/'

In [75]:
len(url_each_coffee)

5746

In [365]:
url = url_each_coffee[0]
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")
coffee_rating = page_soup.findAll("span","review-template-rating")
for rating in coffee_rating:
    print(rating.text)
coffee_roaster = page_soup.findAll("p","review-roaster")
for roaster in coffee_roaster:
    print(roaster.text)
coffee_name = page_soup.findAll("h1","review-title")
for name in coffee_name:
    print(name.text)
coffee_table = page_soup.findAll("table","review-template-table")
for row in coffee_table:
    if "Roaster Location:" in row.find_all('td')[0]:
        print(row.find_all('td')[1].text)
    if "Coffee Origin:" in row.find_all('td')[2]:
        print(row.find_all('td')[3].text)
    if "Roast Level:" in row.find_all('td')[4]:
        print(row.find_all('td')[5].text)
    if "Agtron:" in row.find_all('td')[6]:
        print(row.find_all('td')[7].text)
    if "Est. Price:" in row.find_all('td')[8]:
        print(row.find_all('td')[9].text)
    if "Review Date:" in row.find_all('td')[0]:
        print(row.find_all('td')[1].text)
    if "Aroma:" in row.find_all('td')[2]:
        print(row.find_all('td')[3].text)
    if "Acidity/Structure:" in row.find_all('td')[4]:
        print(row.find_all('td')[5].text)
    if "Body:" in row.find_all('td')[6]:
        print(row.find_all('td')[7].text)
    if "Flavor:" in row.find_all('td')[8]:
        print(row.find_all('td')[9].text)
    if "Aftertaste:" in row.find_all('td')[-2]:
        print(row.find_all('td')[-1].text)
coffee_text = page_soup.findAll("div","review-template")
for name in coffee_text:
    print(name.find_all('p'))

94
Coffee by Design
Costa Rica Naranjo Danilo Salazar Arias
Portland, Maine
Alajuela, Costa Rica
Medium-Light
60/77
$23.00/12 ounces
May 2022
9
9
9							
9
8
[<p class="review-roaster">Coffee by Design</p>, <p><strong>Blind Assessment: </strong>Bright, richly sweet-tart, fruit-toned. Pomegranate, dark chocolate, lilac, almond, amber in aroma and small cup. Bright structure with juicy, vibrant acidity; plush, creamy mouthfeel. Sweetly tart finish with leading notes of pomegranate and almond. Crisply chocolaty, floral finish.
</p>, <p><strong>Notes: </strong>Produced by Danilo Salazar Arias of San Cristobal Farm, from the Catuai, Caturra and Villa Sarchi varieties of Arabica, and processed by the natural method (dried in the whole fruit). Founded in 1994, Coffee By Design is a specialty coffee roaster in Portland, Maine that also operates three coffee shops in town, as well as one in Freeport. Visit www.coffeebydesign.com.

 </p>, <p><strong>The Bottom Line:</strong> A lively, juicy-bri

In [364]:
for row in coffee_table:
    print(row.find_all('td')[-1].text)

$23.00/12 ounces
8


In [366]:
coffee_score = []
coffee_roaster_name = []
coffee_bean_name = []
roaster_location = []
coffee_origin = []
roast_level = []
agtron = []
estimate_price = []
review_date=[]
aroma = []
acidity_or_structure = []
body = []
flavor = []
aftertaste = []
text=[]
for url in url_each_coffee:
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")

    coffee_rating = page_soup.findAll("span","review-template-rating")
    for rating in coffee_rating:
        coffee_score.append(rating.text)

    coffee_roaster = page_soup.findAll("p","review-roaster")
    for roaster in coffee_roaster:
        coffee_roaster_name.append(roaster.text)

    coffee_name = page_soup.findAll("h1","review-title")
    for name in coffee_name:
        coffee_bean_name.append(name.text)

    coffee_table = page_soup.findAll("table","review-template-table")
    for row in coffee_table:
        if "Roaster Location:" in row.find_all('td')[0]:
            roaster_location.append(row.find_all('td')[1].text)

        if "Coffee Origin:" in row.find_all('td')[2]:
            coffee_origin.append(row.find_all('td')[3].text)

        if "Roast Level:" in row.find_all('td')[4]:
            roast_level.append(row.find_all('td')[5].text)

        if "Agtron:" in row.find_all('td')[6]:
            agtron.append(row.find_all('td')[7].text)

        if "Est. Price:" in row.find_all('td')[8]:
            estimate_price.append(row.find_all('td')[9].text)

        if "Review Date:" in row.find_all('td')[0]:
            review_date.append(row.find_all('td')[1].text)

        if "Aroma:" in row.find_all('td')[2]:
            aroma.append(row.find_all('td')[3].text)

        if "Acidity/Structure:" in row.find_all('td')[4]:
            acidity_or_structure.append(row.find_all('td')[5].text)

        if "Body:" in row.find_all('td')[6]:
            body.append(row.find_all('td')[7].text)

        if "Flavor:" in row.find_all('td')[8]:
            flavor.append(row.find_all('td')[9].text)

        if "Aftertaste:" in row.find_all('td')[-2]:
            aftertaste.append(row.find_all('td')[-1].text)

    coffee_text = page_soup.findAll("div","review-template")
    for name in coffee_text:
        text.append(name.find_all('p'))


IndexError: list index out of range

In [369]:
df_columns = ['coffee_score', 'coffee_roaster_name','coffee_bean_name', 'roaster_location', 'coffee_origin' ,\
    'roast_level', 'agtron', 'estimate_price', 'review_date', 'aroma', 'acidity_or_structure', 'body', 'flavor', 'aftertaste', 'text']
coffee_df = pd.DataFrame(columns = df_columns)

row_index = 0
for url in url_each_coffee:
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")

    coffee_rating = page_soup.findAll("span","review-template-rating")
    for rating in coffee_rating:
        coffee_df.loc[row_index, 'coffee_score'] = rating.text

    coffee_roaster = page_soup.findAll("p","review-roaster")
    for roaster in coffee_roaster:
        coffee_df.loc[row_index, 'coffee_roaster_name'] = roaster.text

    coffee_name = page_soup.findAll("h1","review-title")
    for name in coffee_name:
        coffee_df.loc[row_index, 'coffee_bean_name'] = name.text

    coffee_table = page_soup.findAll("table","review-template-table")
    for row in coffee_table:
        try:
            if "Roaster Location:" in row.find_all('td')[0]:
                coffee_df.loc[row_index, 'roaster_location'] = row.find_all('td')[1].text
        except:
            continue

        try:
            if "Coffee Origin:" in row.find_all('td')[2]:
                coffee_df.loc[row_index, 'coffee_origin'] = row.find_all('td')[3].text
        except:
            continue
        
        try:
            if "Roast Level:" in row.find_all('td')[4]:
                coffee_df.loc[row_index, 'roast_level'] = row.find_all('td')[5].text
        except:
            continue

        try:
            if "Agtron:" in row.find_all('td')[6]:
                coffee_df.loc[row_index, 'agtron'] = row.find_all('td')[7].text
        except:
            continue

        try:
            if "Est. Price:" in row.find_all('td')[8]:
                coffee_df.loc[row_index, 'estimate_price'] = row.find_all('td')[9].text
        except:
            continue

        try:
            if "Review Date:" in row.find_all('td')[0]:
                coffee_df.loc[row_index, 'review_date'] = row.find_all('td')[1].text
        except:
            continue

        try:
            if "Aroma:" in row.find_all('td')[2]:
                coffee_df.loc[row_index, 'aroma'] = row.find_all('td')[3].text
        except:
            continue
        
        try:
            if "Acidity/Structure:" in row.find_all('td')[4]:
                coffee_df.loc[row_index, 'acidity_or_structure'] = row.find_all('td')[5].text
        except:
            continue
        
        try:
            if "Body:" in row.find_all('td')[6]:
                coffee_df.loc[row_index, 'body'] = row.find_all('td')[7].text
        except:
            continue
        
        try:
            if "Flavor:" in row.find_all('td')[8]:
                coffee_df.loc[row_index, 'flavor'] = row.find_all('td')[9].text
        except:
            continue
        
        try:
            if "Aftertaste:" in row.find_all('td')[-2]:
                coffee_df.loc[row_index, 'aftertaste'] = row.find_all('td')[-1].text
        except:
            continue
        

    coffee_text = page_soup.findAll("div","review-template")
    for name in coffee_text:
        coffee_df.loc[row_index, 'text'] = name.find_all('p')
    row_index+=1

coffee_df

/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,coffee_score,coffee_roaster_name,coffee_bean_name,roaster_location,coffee_origin,roast_level,agtron,estimate_price,review_date,aroma,acidity_or_structure,body,flavor,aftertaste,text
0,94,Coffee by Design,Costa Rica Naranjo Danilo Salazar Arias,"Portland, Maine","Alajuela, Costa Rica",Medium-Light,60/77,$23.00/12 ounces,May 2022,9,9,9\t\t\t\t\t\t\t,9,8,"[[Coffee by Design], [[Blind Assessment: ], Br..."
1,93,Coffee by Design,Kenya Kahunyo AA,"Portland, Maine","Nyeri growing region, south-central Kenya",Medium-Light,59/76,$22.00/16 ounces,May 2022,9,9,8\t\t\t\t\t\t\t,9,8,"[[Coffee by Design], [[Blind Assessment: ], Cr..."
2,93,Coffee by Design,Honduras COMSA Oscar Omar Alonzo,"Portland, Maine","Marcala, Honduras",Medium-Light,58/74,$18.50/12 ounces,May 2022,9,8,9\t\t\t\t\t\t\t,9,8,"[[Coffee by Design], [[Blind Assessment: ], A ..."
3,92,Coffee by Design,Rwanda Gasharu Natural,"Portland, Maine","Nyamasheke District, Rwanda",Medium-Light,58/76,$23.00/12 ounces,May 2022,8,8,9\t\t\t\t\t\t\t,9,8,"[[Coffee by Design], [[Blind Assessment: ], De..."
4,97,PT's Coffee Roasting Co.,Yemen Haraaz Red Mahal Aqeeq ul Station Natural,"Topeka, Kansas","Sharqi, Haraaz, Yemen",Medium-Light,58/76,$35.00/8 ounces,May 2022,10,9,9\t\t\t\t\t\t\t,10,9,"[[PT's Coffee Roasting Co.], [[Blind Assessmen..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,83,The Coffee Beanery,Beanery Blend,"Flushing, Michigan",NaN,NaN,NaN,NaN,February 1997,8,NaN,7\t\t\t\t\t\t\t,7,NaN,"[[The Coffee Beanery], [[Blind Assessment: ], ..."
5732,81,Starbucks Coffee,House Blend,"Seattle, Washington",NaN,NaN,NaN,NaN,February 1997,7,NaN,6\t\t\t\t\t\t\t,8,NaN,"[[Starbucks Coffee], [[Blind Assessment: ], Th..."
5733,75,Peerless Coffee,President’s Private Blend,"Oakland, California",NaN,NaN,NaN,NaN,February 1997,6,NaN,6\t\t\t\t\t\t\t,5,NaN,"[[Peerless Coffee], [[Blind Assessment: ], Giv..."
5734,74,Gevalia,Traditional Roast,"Des Moines, Iowa",NaN,NaN,NaN,NaN,February 1997,7,NaN,6\t\t\t\t\t\t\t,6,NaN,"[[Gevalia], [[Blind Assessment: ], For such a ..."


# Step 3: Prepare Data for Consumption <a class="anchor" id="Prepare-Data-for-Consumption"></a>

## 3.1 Import Libraries <a class="anchor" id="Import-Libraries"></a>

## 3.11 Load Data Modelling Libraries <a class="anchor" id="Load-Data-Modelling-Libraries"></a>

## 3.2 Meet and Greet Data <a class="anchor" id="Meet-and-Greet-Data"></a>

1)The FGM variable is outcome or dependent variable. It is a binary nominal datatype of 1 for make and 0 for missed. 
All other variables are potential or independent variables. 
Its important to note, more predictor variables do not make a better model, but the right variables.
<br>
2)The GameID, match, win, Final_margin, shot_number, and PTS are assumed to be random unique identifiers, that have no impact on the outcome veribale. Thus, they will be excluded from analysis.
<br>
4)The Name and shot_number variable are nominal datatype. It could be used in feature engineering to derive the who the best defender is, the hot-hand hypothesis, etc. Since these variables already exist, we'll make use of it to see if player makes a difference.
<br>
5)The Location and PTS_type variables are a nominal datatype. They will be converted to dummy variables for mathematical calculations.
<br>
6)The game_clock, shot_clock, dribbles,  touch_time, shot_distance, and closet_defender_distance variable are continuous quantitative datatypes.


### 3.21 The 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting <a class="anchor" id="4C"></a>

### 3.22 Clean Data <a class="anchor" id="Clean-Data"></a>

### 3.23 Convert Formats <a class="anchor" id="Convert-Formats"></a>

### 3.24 Da-Double Check Cleaned Data <a class="anchor" id="Da-Double-Check-Cleaned-Data"></a>

### 3.25 Split Training and Testing Data <a class="anchor" id="Split-Training-and-Testing-Data"></a>

# Step 4: Perform Exploratory Analysis with Statistics <a class="anchor" id="Perform-Exploratory-Analysis-with-Statistics"></a>

# Step 5: Model Data <a class="anchor" id="Model-Data"></a>

## 5.1 Evaluate Model Performance <a class="anchor" id="Evaluate-Model-Performance"></a>

### 5.11 Model Performance with Cross-Validation (CV) <a class="anchor" id="CV"></a>

### 5.12 Tune Model with Hyper-Parameters <a class="anchor" id="Tune-Model-with-Hyper-Parameters"></a>

### 5.13 Tune Model with Feature Selection <a class="anchor" id="Tune-Model-with-Feature-Selection"></a>

# Step 6: Validate and Implement <a class="anchor" id="Validate-and-Implement"></a>

# Step 7: Optimize and Strategize<a class="anchor" id="Optimize-and-Strategize"></a>

### Conclusion
Iteration one of the Data Science Framework, seems to converge on 0.77990 submission accuracy. Using the same dataset and different implementation of a random forest (adaboost,decision tree, gradient boost, xgboost, etc.) with tuning does not exceed the 0.77990 submission accuracy. Interesting for this dataset, the simple random forest algorithm had the best default submission score and with tuning achieved the same best accuracy score.

While no general conclusions can be made from testing a handful of algorithms on a single dataset, there are several observations on the mentioned dataset.

The train dataset has a different distribution than the test/validation dataset and population. This created wide margins between the cross validation (CV) accuracy score .
Given the same dataset, random forest based algorithms, seemed to converge on the same accuracy score after proper tuning.
Despite tuning, no machine learning algorithm, exceeded the homemade algorithm. The author will theorize, that for small datasets, a manmade algorithm is the bar to beat.
With that in mind, for iteration two, I would spend more time on preprocessing and feature engineering. In order to better align the CV score and improve the overall accuracy.